In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns



In [45]:
data = pd.read_csv('../data/datadaily/data_daily0.csv', index_col='Date_time_nr')
Xdata = data.drop(columns=['P_avg'])
ydata = data[['P_avg']]

In [46]:
data.head()
Xdata.head()
#ydata.head()

,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h
Date_time_nr,,,,,,,
1356998400,5.314167,1012.458333,88.083333,5.037083,212.500000,0.332917,0.0
1357084800,2.317500,1027.291667,93.625000,3.236250,222.500000,0.000000,0.0
1357171200,4.582500,1037.041667,95.166667,5.037083,209.583333,0.041667,0.0
1357257600,7.101250,1037.791667,97.666667,2.915417,246.250000,0.000000,0.0
1357344000,5.635000,1034.791667,94.166667,2.014167,232.083333,0.000000,0.0


In [47]:
jdata = pd.read_json('../data/json/jdata1.json', orient='split')

In [48]:
jdata.head()

,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h
2013-01-01,5.314167,1012.458333,88.083333,5.037083,212.500000,0.332917,0
2013-01-02,2.317500,1027.291667,93.625000,3.236250,222.500000,0.000000,0
2013-01-03,4.582500,1037.041667,95.166667,5.037083,209.583333,0.041667,0
2013-01-04,7.101250,1037.791667,97.666667,2.915417,246.250000,0.000000,0
2013-01-05,5.635000,1034.791667,94.166667,2.014167,232.083333,0.000000,0


# ML

In [42]:
from sklearn.preprocessing import MinMaxScaler

def ML_scaler(data, scaler=None):
    if scaler == None:
        return data
    cols = data.columns
    index = data.index
    data_sc = pd.DataFrame(data =  scaler.fit_transform(data), columns=cols, index=index)
    return data_sc

def add_yearly_fourier(df):
    # timestamps as date
    ts = pd.to_datetime(df.index, unit='s')
    # Day of year (1..365 or 366 for leap)
    day_of_year = ts.dayofyear
    # Period T = 365 days (approx yearly cycle)
    T = 365
    # Add Fourier components
    df["year_sin"] = np.sin(2 * np.pi * day_of_year / T)
    df["year_cos"] = np.cos(2 * np.pi * day_of_year / T)
    return df

def encode_wind_direction_circular(df):
    """
    Add smooth circular (sin/cos) encoding for wind direction in degrees (0–360).
    """
    # Normalize angle to 0–360
    theta = np.deg2rad(df['wind_deg'] % 360)
    df["wind_sin"] = np.sin(theta)
    df["wind_cos"] = np.cos(theta)
    df.drop(columns=['wind_deg'], inplace=True)
    return df
    
def pipeline(Xdata):
    scaler = MinMaxScaler()
    data = ML_scaler(Xdata, scaler)
    data = encode_wind_direction_circular(data)
    data = add_yearly_fourier(data)

    return data


In [ ]:
#encode_wind_direction_circular(jdata)
#add_yearly_fourier(jdata)
#scaler = MinMaxScaler()
#ML_scaler(Xdata, scaler)

In [50]:
pipeline(Xdata)

,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,wind_sin,wind_cos,year_sin,year_cos
Date_time_nr,,,,,,,,,,
1356998400,0.320989,0.514364,0.806364,0.302514,0.182670,0.000000,0.011000,0.999939,1.721336e-02,0.999852
1357084800,0.235290,0.757866,0.896412,0.166871,0.000000,0.000000,0.011547,0.999933,3.442161e-02,0.999407
1357171200,0.300064,0.917921,0.921462,0.302514,0.022862,0.000000,0.010840,0.999941,5.161967e-02,0.998667
1357257600,0.372096,0.930233,0.962085,0.142705,0.000000,0.000000,0.012847,0.999917,6.880243e-02,0.997630
1357344000,0.330164,0.880985,0.905213,0.074820,0.000000,0.000000,0.012072,0.999927,8.596480e-02,0.996298
...,...,...,...,...,...,...,...,...,...,...
1514332800,0.304354,0.127223,0.827353,0.454320,0.159122,0.000000,0.010886,0.999941,-6.880243e-02,0.997630
1514419200,0.202664,0.380985,0.813135,0.265418,0.000000,0.000000,0.013166,0.999913,-5.161967e-02,0.998667
1514505600,0.226627,0.442544,0.813135,0.426733,0.005258,0.135314,0.010430,0.999946,-3.442161e-02,0.999407


In [51]:
pipeline(jdata)

,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,wind_sin,wind_cos,year_sin,year_cos
2013-01-01,0.707375,0.000000,0.000000,1.000000,1.000000,0.0,0.013312,0.999911,0.017213,0.999852
2013-01-02,0.216688,0.585526,0.578261,0.484679,0.000000,0.0,0.014539,0.999894,0.034422,0.999407
2013-01-03,0.587569,0.970395,0.739130,1.000000,0.125156,0.0,0.012954,0.999916,0.051620,0.998667
2013-01-04,1.000000,1.000000,1.000000,0.392870,0.000000,0.0,0.017452,0.999848,0.068802,0.997630
2013-01-05,0.759910,0.881579,0.634783,0.134971,0.000000,0.0,0.015714,0.999877,0.085965,0.996298
2013-01-06,0.652930,0.768092,0.800000,0.124359,0.000000,0.0,0.016456,0.999865,0.103102,0.994671
2013-01-07,0.488845,0.631579,0.491304,0.000000,0.000000,0.0,0.000000,1.000000,0.120208,0.992749
2013-01-08,0.187897,0.608553,0.878261,0.110290,0.000000,0.0,0.007842,0.999969,0.137279,0.990532
2013-01-09,0.000000,0.407895,0.921739,0.253130,0.053817,0.0,0.011369,0.999935,0.154309,0.988023
2013-01-10,0.310500,0.162829,0.878261,0.723620,0.168961,0.0,0.014590,0.999894,0.171293,0.985220
